In [ ]:

# r = model.to(device)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# from datasets import load_dataset
# import seaborn as sns
from tqdm import tqdm
import numpy as np
import transformers
from baukit import TraceDict
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/tnief/miniconda3/envs/wiki-probes/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.cuda.empty_cache()
torch.cuda.set_device(0) # Sets the default device for tensors to be the first GPU.
device = "cuda:0"
torch.cuda.device_count()

1

In [5]:
# Load the model

# MODEL = "/net/projects/veitch/LLMs/llama2-based-models/llama2-hf/Llama-2-7b-chat-hf"
MODEL = "/net/projects/veitch/LLMs/llama1-based-models/alpaca-7b"

tokenizer = transformers.LlamaTokenizer.from_pretrained(MODEL)
model = transformers.LlamaForCausalLM.from_pretrained(MODEL, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto").to(device)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Loading checkpoint shards: 100%|██████████| 3/3 [02:06<00:00, 42.20s/it]


In [6]:
# Any prompt will do to demonstrate. This prompt is 82 tokens long.

template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please say yes only if it costs between {:.2f} and {:.2f} dollars, otherwise no.

### Input:
{:.2f} dollars

### Response:
"""

prompt = template.format(3.22,5.76,9.30)
print(prompt)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please say yes only if it costs between 3.22 and 5.76 dollars, otherwise no.

### Input:
9.30 dollars

### Response:



In [7]:
# Just generate some prompts for demonstration
# You'd probably save a dataset of prompts, and load those from a file.

def generate_prompts(template,n=1000,include_bounds=False):
    "Replicates the same distribution as BDAS paper."
    for i in range(n):
        # Generate the lower bound, upper bound, and input value
        lower_bound = np.round(np.random.uniform(0.00,7.49),2)
        max_ub = np.min([lower_bound+7.5,9.99])
        upper_bound = np.round(np.random.uniform(lower_bound+2.5,max_ub),2)
        diff = np.round(upper_bound - lower_bound,2)
        assert 2.5 <= diff and diff <= 7.5, (lower_bound, max_ub, upper_bound, diff)
        input_value = np.round(np.random.uniform(0.00,9.99),2)

        # Generate the prompt
        prompt = template.format(lower_bound,upper_bound,input_value)
        if include_bounds:
            yield (lower_bound,upper_bound,input_value,prompt)
        else:
            yield prompt

prompts = [p for p in generate_prompts(template,n=10)]

In [8]:
def get_activations(prompts,tokenizer,model,device,layer="all"):
    """Returns a Numpy array of residual stream activations. 
    Based on https://github.com/likenneth/honest_llama
    
    David's uncertainties: I think these are the activations before the MLP sublayer?
    """
    # input_ids = tokenizer(prompts, padding=True, truncation=True, return_tensors="pt").input_ids.to(device)
    # attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.to(device)

    tokenized = tokenizer(prompts, padding=True, truncation=True, return_tensors="pt")
    input_ids = tokenized.input_ids.to(device)
    attention_mask = tokenized.attention_mask.to(device)

    # print(f"Model on: {model.device if hasattr(model, 'device') else next(model.parameters()).device}")
    # print(f"Input IDs on: {input_ids.device}")
    # print(f"Attention Mask on: {attention_mask.device}")

    # print(tokenizer.convert_ids_to_tokens(tokenized["input_ids"][0]))

    model.eval()
    outputs = model(
        input_ids,
        attention_mask=attention_mask, output_hidden_states = True
    )
    hidden_states = outputs.hidden_states
    if layer == "all":
         # (num_layers, batch_size, seq_length, hidden_dim)
        hidden_states = torch.stack(hidden_states, dim = 0).squeeze()
        hidden_states = hidden_states.detach().cpu().numpy()
    else:
         # (batch_size, seq_length, hidden_dim)
        hidden_states = hidden_states[layer].detach().cpu().numpy()
    return hidden_states

In [9]:
# Single prompt, layer 15
hidden_states = get_activations(prompts[:1],tokenizer,model,device,layer=15)
print(hidden_states.shape)

(1, 82, 4096)


In [10]:
# Single prompt, all layers. 
# Note that in this case the shape drops the singluar batch_size dimension. Maybe we should adjust this behavior. But our use case is probing, which is multiple prompts.
hidden_states = get_activations(prompts[:1],tokenizer,model,device)
print(hidden_states.shape)

(33, 82, 4096)


In [11]:
# Multiple prompt, layer 15
hidden_states = get_activations(prompts,tokenizer,model,device,layer=15)
print(hidden_states.shape)

(10, 82, 4096)


In [12]:
# Multiple prompt, all layers
hidden_states = get_activations(prompts,tokenizer,model,device)
print(hidden_states.shape)

(33, 10, 82, 4096)


If you have a supervised dataset of (prompts,labels), then to train a probe, you'll just replace the prompts with the activations in the learning objective. So you'll train a classifier on (activations,labels).

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
from datasets import Dataset
from sklearn.preprocessing import MultiLabelBinarizer
from torch.nn import BCEWithLogitsLoss

In [14]:
LABELS = "title"

In [15]:
df_train = pd.read_csv("data/parsed-paragraphs-train.csv")
df_test = pd.read_csv("data/parsed-paragraphs-test.csv")

In [16]:
# TODO: hacky way to get unique labels - set up pipeline to actually handle this well

if LABELS == "categories":
    from ast import literal_eval

    unique_labels = set()
    for item in df_train['label_list'].tolist():
        for sub_item in item:
            labels = literal_eval(sub_item)
            unique_labels.update(labels)

    unique_labels = list(unique_labels)
elif LABELS == "title":
    unique_labels = list(df_train['title'].drop_duplicates())

In [17]:
unique_labels

['Anarchism',
 'Albedo',
 'A',
 'Alabama',
 'Achilles',
 'Abraham Lincoln',
 'Aristotle',
 'An American in Paris',
 'Academy Award for Best Production Design',
 'Academy Awards',
 'Animalia (book)',
 'International Atomic Time',
 'Altruism',
 'Ayn Rand',
 'Alain Connes',
 'Allan Dwan',
 'Algeria',
 'List of Atlas Shrugged characters',
 'Anthropology',
 'Agricultural science']

In [18]:
def df_to_dataloader(df, model, labels=LABELS, save=False):
    if labels=="title":
        unique_labels = list(df_train['title'].drop_duplicates())
        label_encoder = LabelEncoder()
        df['label_encoded'] = label_encoder.fit_transform(df['title'])
    elif labels=="categories":
        df['label_list'] = df['label'].apply(lambda x: x.split('|'))
        unique_labels = set()
        for item in df_train['label_list'].tolist():
            for sub_item in item:
                labels = literal_eval(sub_item)
                unique_labels.update(labels)

        unique_labels = list(unique_labels)

        label_encoder = LabelEncoder()
        label_encoder.fit(unique_labels)

        encoded_labels = []
        for item in df_train['label_list'].tolist():
            for sub_item in item:
                labels = literal_eval(sub_item)
                encoded_labels.append(label_encoder.transform(labels).tolist())
        df['label_encoded'] = encoded_labels

        def list_to_binary_vector(lst, dim=len(unique_labels)):
            return [1 if i in lst else 0 for i in range(dim)]

        # Assuming df['label_encoded'] is already a list of integers
        df['binary_labels'] = df['label_encoded'].apply(list_to_binary_vector)

    Xs = []
    ys = []
    for i, row in df.iterrows():
        hidden_states = get_activations(row.text,tokenizer,model,device)
        Xs.append(hidden_states[:,0,:]) # take only the <s> token
        if labels == "categories":
            ys.append(row.binary_labels)
        elif labels == "title":
            ys.append(row.label_encoded)
    
    Xs_t = Tensor(Xs).float()
    ys_t = Tensor(np.asarray(ys)).float() if labels == "categories" else Tensor(np.asarray(ys)).long()

    tensor_dataset = TensorDataset(Xs_t, ys_t)

    if save:
        torch.save(tensor_dataset, 'data/example_dataset.pt')

    return DataLoader(tensor_dataset, batch_size=4, shuffle=True)

In [19]:
train_loader = df_to_dataloader(df_train, model)
test_loader = df_to_dataloader(df_test, model)

/net/scratch/tnief/tmp/ipykernel_3105227/2820583812.py:42: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  Xs_t = Tensor(Xs).float()


In [20]:
class Linear(nn.Module):
    def __init__(self, n_classes=len(unique_labels)):
        super().__init__()
        self.fc1 = nn.Linear(33*4096, n_classes)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

In [21]:
class MLP(nn.Module):
    def __init__(self, n_classes=len(unique_labels)):
        super().__init__()
        self.fc1 = nn.Linear(33*4096, 120)
        self.fc2 = nn.Linear(120, n_classes)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [22]:
import torch.nn.init as init

def init_weights(m):
    if type(m) == nn.Linear:
        init.xavier_normal_(m.weight)
        init.constant_(m.bias, 0)

In [23]:
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=.5, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

In [34]:
linear = Linear()
mlp = MLP()
probe = mlp
probe.apply(init_weights)

MLP(
  (fc1): Linear(in_features=135168, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=20, bias=True)
)

In [35]:
import torch.optim as optim

if LABELS == "catgories":
    criterion = BCEWithLogitsLoss(pos_weight=Tensor(torch.ones(len(unique_labels)) * 20))
elif LABELS == "title":
    criterion = nn.CrossEntropyLoss()

# optimizer = optim.AdamW(probe.parameters(), lr=0.001)
optimizer = optim.SGD(probe.parameters(), lr=.001, momentum=)

In [36]:
predicted_labels = 0
for epoch in range(50):  
    probe.train()
    train_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = probe(inputs)
        if LABELS == "categories":
            pos_labels = labels.sum().item()
            neg_labels = labels.numel() - pos_labels
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    if (epoch + 1) % 5 == 0:
        val_loss = 0.0
        val_count = 0
        predicted_labels = 0
        probe.eval()
        with torch.no_grad():
            for i, data in enumerate(test_loader):  
                inputs, labels = data
                outputs = probe(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_count += 1
                if LABELS == "categories":
                    predicted_labels += (torch.sigmoid(outputs) > .5).int().sum().item()

        print(f'[Training][{epoch + 1}] loss: {train_loss / len(train_loader):.3f}')
        print(f'[Validation][{epoch + 1}] loss: {val_loss / val_count:.3f}')
        if LABELS == "categories":
            print(f'[Predicted Validation Labels]: {predicted_labels}')

[Training][5] loss: 2.996
[Validation][5] loss: 2.996
[Training][10] loss: 2.996
[Validation][10] loss: 2.996
[Training][15] loss: 2.996
[Validation][15] loss: 2.996
[Training][20] loss: 2.996
[Validation][20] loss: 2.996
[Training][25] loss: 2.996
[Validation][25] loss: 2.996
[Training][30] loss: 2.996
[Validation][30] loss: 2.996
[Training][35] loss: 2.996
[Validation][35] loss: 2.996
[Training][40] loss: 2.996
[Validation][40] loss: 2.996
[Training][45] loss: 2.996
[Validation][45] loss: 2.996
[Training][50] loss: 2.996
[Validation][50] loss: 2.996


In [31]:
data, labels = next(iter(test_loader))

In [38]:
for i, (inputs, labels) in enumerate(train_loader):
    print(f"Batch {i+1}")
    print("Labels:", labels)

Batch 1
Labels: tensor([ 8,  2, 16,  5])
Batch 2
Labels: tensor([ 3, 12, 12,  6])
Batch 3
Labels: tensor([ 3, 13,  8,  1])
Batch 4
Labels: tensor([11, 17,  5, 17])
Batch 5
Labels: tensor([1, 6, 2, 9])
Batch 6
Labels: tensor([11, 18, 14, 19])
Batch 7
Labels: tensor([ 0, 18,  9,  4])
Batch 8
Labels: tensor([ 7, 13,  4,  7])
Batch 9
Labels: tensor([14, 10, 16, 15])
Batch 10
Labels: tensor([ 0, 15, 19, 10])


In [32]:
labels

tensor([ 0, 15, 14, 11])

In [33]:
_, predicted_labels = torch.max(probe(data), dim=1)
predicted_labels

tensor([17, 17, 17, 17])

In [46]:
labels

tensor([13, 19, 16, 12])

In [ ]:
(torch.sigmoid(mlp(data))[0] > .5).int().sum().item()

In [ ]:
(torch.sigmoid(mlp(data))[0] > .5).int().item()